# LOFAR scintillation data


### We will inspect beamformed data of single stations pointing at one of the brightest radio sources in the norther hemisphere

First some basic imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

The data format we will work with is either hdf5 or fits. Fits files can easily be read using the astropy.io.fits package. For the hdf5 data we use h5py

In [ ]:
import h5py
from astropy.io import fits

The raw LOFAR beamformed data is generally stored in hdf5 format. Depending on how much averaging has happened before storing the data, these files can become very large. Let's start by inspecting one of the hdf5 files.  

In [ ]:
myfile = h5py.File("test_data_CasA.h5")

An hdf5 file contains data arrays and metadata in a tree like structure. myfile is pointing at the root of the file. You can access the branches below the root by checking the keys:

In [ ]:
print (list(myfile.keys()))
rootkeys = list(myfile.keys())
print(list(myfile[rootkeys[0]].keys()))

The metadata can be accessed similarly via the attrs function. 

In [ ]:
print (list(myfile.attrs))

##### Excercise

Try to familiarize yourself with the dataformat. What is the time resolution of the data? How many stations do we have data for?